# Preparação da Base de Dados Para o Treinamento do Modelo
Esse caderno contêm os passos para a prepação das bases de dados

O resultado final é uma tabela com as *features* e a predição da nota do ideb dois anos depois

Essa tabela final é uma tabela praticamente pronta para ser utilizada para treinar algum modelo de *machine learning*

In [1]:
# import libs and opotions

import re
import warnings
import pandas as pd
import sklearn
import numpy as np
import gc

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
warnings.filterwarnings('ignore')

def to_float(x):
   return float(re.sub("\,", ".", str(x)))

In [2]:
# Loading datasets

# escolas datasets
escola2007 = pd.read_csv('datasets_bcg/escolas20070101.csv', na_values='-')
escola2009 = pd.read_csv('datasets_bcg/escolas20090101.csv', na_values='-')
escola2011 = pd.read_csv('datasets_bcg/escolas20110101.csv', na_values='-')
escola2013 = pd.read_csv('datasets_bcg/escolas20130101.csv', na_values='-')
escola2015 = pd.read_csv('datasets_bcg/escolas20150101.csv', na_values='-')
escola2017 = pd.read_csv('datasets_bcg/escolas20170101.csv', na_values='-')
escola2007['cod_escola'] = escola2007['cod_escola_inep'].astype(int)
escola2009['cod_escola'] = escola2009['cod_escola_inep'].astype(int)
escola2011['cod_escola'] = escola2011['cod_escola_inep'].astype(int)
escola2013['cod_escola'] = escola2013['cod_escola_inep'].astype(int)
escola2015['cod_escola'] = escola2015['cod_escola_inep'].astype(int)
escola2017['cod_escola'] = escola2017['cod_escola_inep'].astype(int)
dframes = [escola2007,escola2009,escola2011,escola2013,escola2015,escola2017]
df_escolas = pd.concat(dframes)

#ideb escolas dataset
df_ideb_escolas = pd.read_csv('datasets_bcg/ideb_escolas_anosfinais2005_2017.csv', encoding = "ISO-8859-1",na_values='-')
df_ideb_escolas['cod_escola'] = df_ideb_escolas['Cod_Escola_Completo'].astype(int)

# merge escolas with ideb_escolas
df_full = pd.merge(df_escolas, df_ideb_escolas, how='left', on='cod_escola')

# pib dataset
df_pib_cidades = pd.read_csv('pib/pib_cidades.csv', encoding = "ISO-8859-1",na_values='-')
df_pib_cidades['pib'] = df_pib_cidades['pib'].apply(to_float)


# pop. Cidades
df_pop_cidades = pd.read_excel('população_ibge/população.xlsx', encoding = "ISO-8859-1",na_values='-')
df_pop_cidades['pop'] = df_pop_cidades['pop'].apply(to_float)

# merge with pib and pop dataset
df_full['municipio_ano'] = df_full['cod_municipio'] * 10000 + df_full['ano_censo']
df_full = pd.merge(df_full, df_pib_cidades, how='left', on='municipio_ano')
df_full = pd.merge(df_full, df_pop_cidades, how='left', on='municipio_ano')
df_full

,Unnamed: 0,ano_censo,cod_escola_inep,regiao,unidade_federativa,mesoregiao,microregiao,municipio,tp_dependencia,tp_dependencia_desc,tp_rede_publica,rede_publica_desc,in_somente_matriculas_ac_ou_aee,tp_rede,tp_rede_nome,tp_rede_cod,nome_escola,num_matriculas,num_estudantes,in_comum_creche,in_comum_pre,in_comum_fund_ai,in_comum_fund_af,in_comum_medio_medio,in_comum_medio_integrado,in_comum_medio_normal,num_estudantes_ensino_infantil,num_estudantes_ensino_fund_anos_iniciais,num_estudantes_ensino_fund_anos_finais,num_estudantes_ensino_medio_propedeutico,num_estudantes_fund_1_ano,num_estudantes_fund_2_ano,num_estudantes_fund_3_ano,num_estudantes_fund_4_ano,num_estudantes_fund_5_ano,num_estudantes_fund_6_ano,num_estudantes_fund_7_ano,num_estudantes_fund_8_ano,num_estudantes_fund_9_ano,num_estudantes_medio_1_serie,num_estudantes_medio_2_serie,num_estudantes_medio_3_serie,num_estudantes_medio_4_serie,num_estudantes_medio_nao_seriada,num_turmas,tp_situacao_funcionamento,tp_situacao_funcionamento_desc,in_em_atividade,tp_localizacao,tp_localizacao_desc,tp_categoria_escola_privada,tp_categoria_escola_privada_desc,in_conveniada_pp,in_conveniada_pp_desc,tp_convenio_poder_publico,tp_convenio_poder_publico_desc,in_local_func_predio_escolar,tp_ocupacao_predio_escolar,tp_ocupacao_predio_escolar_desc,in_local_func_salas_empresa,in_local_func_socioeducativo,in_local_func_unid_prisional,in_local_func_prisional_socio,in_local_func_templo_igreja,in_local_func_casa_professor,in_local_func_galpao,tp_ocupacao_galpao,tp_ocupacao_galpao_desc,in_local_func_salas_outra_esc,in_local_func_outros,in_predio_compartilhado,in_agua_filtrada,in_agua_rede_publica,in_agua_poco_artesiano,in_agua_cacimba,in_agua_fonte_rio,in_agua_inexistente,in_energia_rede_publica,in_energia_gerador,in_energia_outros,in_energia_inexistente,in_esgoto_rede_publica,in_esgoto_fossa,in_esgoto_inexistente,in_lixo_coleta_periodica,in_lixo_queima,in_lixo_joga_outra_area,in_lixo_recicla,in_lixo_enterra,in_lixo_outros,in_sala_diretoria,in_sala_professor,in_laboratorio_informatica,in_laboratorio_ciencias,in_sala_atendimento_especial,in_quadra_esportes_coberta,in_quadra_esportes_descoberta,in_quadra_esportes,in_cozinha,in_biblioteca,in_sala_leitura,in_biblioteca_sala_leitura,in_parque_infantil,in_bercario,in_banheiro_fora_predio,in_banheiro_dentro_predio,in_banheiro_ei,in_banheiro_pne,in_dependencias_pne,in_secretaria,in_banheiro_chuveiro,in_refeitorio,in_despensa,in_almoxarifado,in_auditorio,in_patio_coberto,in_patio_descoberto,in_alojam_aluno,in_alojam_professor,in_area_verde,in_lavanderia,in_dependencias_outras,num_salas_existentes,num_salas_utilizadas,in_equip_tv,in_equip_videocassete,in_equip_dvd,in_equip_parabolica,in_equip_copiadora,in_equip_retroprojetor,in_equip_impressora,in_equip_impressora_mult,in_equip_som,in_equip_multimidia,in_equip_fax,in_equip_foto,in_computador,num_equip_tv,num_equip_videocassete,num_equip_dvd,num_equip_parabolica,num_equip_copiadora,num_equip_retroprojetor,num_equip_impressora,num_equip_impressora_mult,num_equip_som,num_equip_multimidia,num_equip_fax,num_equip_foto,num_computador,num_comp_administrativo,num_comp_aluno,in_internet,in_banda_larga,num_funcionarios,in_alimentacao,num_professores,num_professores_em_regencia,num_professores_em_regencia_fund_ai,num_professores_em_regencia_fund_af,num_professores_em_regencia_ens_medio,ideb_2013_ai_media_port,ideb_2013_ai_media_mat,ideb_2013_ai_proficiencia,ideb_2013_ai_fluxo,ideb_2013_anos_iniciais,ideb_2013_af_media_port,ideb_2013_af_media_mat,ideb_2013_af_proficiencia,ideb_2013_af_fluxo,ideb_2013_anos_finais,ideb_2015_ai_media_port,ideb_2015_ai_media_mat,ideb_2015_ai_proficiencia,ideb_2015_ai_fluxo,ideb_2015_anos_iniciais,ideb_2015_af_media_port,ideb_2015_af_media_mat,ideb_2015_af_proficiencia,ideb_2015_af_fluxo,ideb_2015_anos_finais,tp_aee,tp_aee_desc,tp_atividade_complementar,tp_atividade_complementar_desc,in_fundamental_ciclos,tp_localizacao_diferenciada,tp_localizacao_diferenciada_desc,in_material_esp_quilombola

In [3]:
#chosing columns from the datasets

df_full_filtered_cols_2007_2017 = df_full[['ano_censo','cod_escola','cod_municipio','municipio','regiao','unidade_federativa','Rede','pib', 'pop' ,'num_estudantes_ensino_fund_anos_finais','num_professores_em_regencia_fund_af', 'in_agua_filtrada', 'in_agua_inexistente', 'in_energia_rede_publica', 'in_energia_inexistente', 'in_esgoto_rede_publica', 'in_esgoto_inexistente', 'in_sala_professor', 'in_laboratorio_informatica', 'in_laboratorio_ciencias', 'in_sala_atendimento_especial', 'in_quadra_esportes', 'in_cozinha', 'in_biblioteca', 'in_sala_leitura', 'in_biblioteca_sala_leitura', 'in_parque_infantil', 'in_bercario', 'in_banheiro_fora_predio', 'in_banheiro_dentro_predio', 'in_secretaria', 'in_refeitorio', 'in_auditorio', 'in_patio_coberto', 'in_patio_descoberto', 'in_alojam_aluno', 'in_alojam_professor', 'in_area_verde', 'in_lavanderia','num_salas_utilizadas',"num_equip_tv", "num_equip_videocassete", "num_equip_dvd", "num_equip_parabolica", "num_equip_copiadora", "num_equip_retroprojetor", "num_equip_impressora", "num_equip_impressora_mult", "num_equip_som", "num_equip_multimidia", "num_equip_fax", "num_equip_foto", "num_computador", "num_comp_administrativo", "num_comp_aluno", "in_internet","num_funcionarios", 'NotaProvaBrasil_MT_2005','NotaProvaBrasil_LP_2005','NotaProvaBrasil_NotaMedia_2005','NotaProvaBrasil_MT_2007','NotaProvaBrasil_LP_2007','NotaProvaBrasil_NotaMedia_2007', 'NotaProvaBrasil_MT_2009','NotaProvaBrasil_LP_2009','NotaProvaBrasil_NotaMedia_2009','NotaProvaBrasil_MT_2011','NotaProvaBrasil_LP_2011','NotaProvaBrasil_NotaMedia_2011','NotaProvaBrasil_MT_2013','NotaProvaBrasil_LP_2013','NotaProvaBrasil_NotaMedia_2013','NotaProvaBrasil_MT_2015','NotaProvaBrasil_LP_2015','NotaProvaBrasil_NotaMedia_2015','NotaProvaBrasil_MT_2017','NotaProvaBrasil_LP_2017','NotaProvaBrasil_NotaMedia_2017','IndicadorRendimento_2005','IndicadorRendimento_2007','IndicadorRendimento_2009','IndicadorRendimento_2011','IndicadorRendimento_2013','IndicadorRendimento_2015','IndicadorRendimento_2017','Ideb2005','Ideb2007','Ideb2009','Ideb2011','Ideb2013','Ideb2015','Ideb2017']]

In [4]:
# free memory
del escola2007
del escola2009
del escola2011
del escola2013
del escola2015
del escola2017
del df_ideb_escolas
del df_escolas
del df_full
del df_pib_cidades
gc.collect()

23706

In [5]:
# splitting datasets on 2015 because we dont have real observations from 2019 so 2017 cant be used to train

df_full_filtered_cols_2007_2015 = df_full_filtered_cols_2007_2017[(df_full_filtered_cols_2007_2017.ano_censo < 2016)]

In [6]:
# creating columns, preparing data and removing columns for the final df

# x

df_final_2007_2015 = df_full_filtered_cols_2007_2015.copy()
# convert boolean to 0 and 1
df_final_2007_2015[['in_agua_filtrada', 'in_agua_inexistente', 'in_energia_rede_publica', 'in_energia_inexistente', 'in_esgoto_rede_publica', 'in_esgoto_inexistente', 'in_sala_professor', 'in_laboratorio_informatica', 'in_laboratorio_ciencias', 'in_sala_atendimento_especial', 'in_quadra_esportes', 'in_cozinha', 'in_biblioteca', 'in_sala_leitura', 'in_biblioteca_sala_leitura', 'in_parque_infantil', 'in_bercario', 'in_banheiro_fora_predio', 'in_banheiro_dentro_predio', 'in_secretaria', 'in_refeitorio', 'in_auditorio', 'in_patio_coberto', 'in_patio_descoberto', 'in_alojam_aluno', 'in_alojam_professor', 'in_area_verde', 'in_lavanderia', 'in_internet']] =  df_final_2007_2015[['in_agua_filtrada', 'in_agua_inexistente', 'in_energia_rede_publica', 'in_energia_inexistente', 'in_esgoto_rede_publica', 'in_esgoto_inexistente', 'in_sala_professor', 'in_laboratorio_informatica', 'in_laboratorio_ciencias', 'in_sala_atendimento_especial', 'in_quadra_esportes', 'in_cozinha', 'in_biblioteca', 'in_sala_leitura', 'in_biblioteca_sala_leitura', 'in_parque_infantil', 'in_bercario', 'in_banheiro_fora_predio', 'in_banheiro_dentro_predio', 'in_secretaria', 'in_refeitorio', 'in_auditorio', 'in_patio_coberto', 'in_patio_descoberto', 'in_alojam_aluno', 'in_alojam_professor', 'in_area_verde', 'in_lavanderia', 'in_internet']].astype(bool).replace(True,1).replace(False,0).astype(int)
# feature: numero de alunos por numero de professores
df_final_2007_2015['aluno_prof_ratio'] = df_final_2007_2015['num_estudantes_ensino_fund_anos_finais'] / df_final_2007_2015['num_professores_em_regencia_fund_af'].replace(0, 1) # to not divide by zero
# get only the values on columns matching the current year 
df_final_2007_2015['NotaProvaBrasil_MT_current_year'] = [df_final_2007_2015['NotaProvaBrasil_MT_' + str(ano)][i] for i, ano in enumerate(df_final_2007_2015['ano_censo'])]
df_final_2007_2015['NotaProvaBrasil_LP_current_year'] = [df_final_2007_2015['NotaProvaBrasil_LP_' + str(ano)][i] for i, ano in enumerate(df_final_2007_2015['ano_censo'])]
df_final_2007_2015['NotaProvaBrasil_NotaMedia_current_year'] = [df_final_2007_2015['NotaProvaBrasil_NotaMedia_' + str(ano)][i] for i, ano in enumerate(df_final_2007_2015['ano_censo'])]
df_final_2007_2015['IndicadorRendimento__currrent_year'] = [df_final_2007_2015['IndicadorRendimento_' + str(ano)][i] for i, ano in enumerate(df_final_2007_2015['ano_censo'])]
df_final_2007_2015['Ideb_current_year'] = [df_final_2007_2015['Ideb' + str(ano)][i] for i, ano in enumerate(df_final_2007_2015['ano_censo'])]
# foracasting lag and diff
df_final_2007_2015['Ideb_past_2_years'] = [df_full_filtered_cols_2007_2017['Ideb' + str(ano - 2)][i] for i, ano in enumerate(df_final_2007_2015['ano_censo'])]
df_final_2007_2015['Ideb_diff'] = df_final_2007_2015['Ideb_current_year'] - df_final_2007_2015['Ideb_past_2_years']

# y

df_final_2007_2015['Ideb_next_2_years'] = [df_full_filtered_cols_2007_2017['Ideb' + str(ano + 2)][i] for i, ano in enumerate(df_final_2007_2015['ano_censo'])]

#removing other columns
df_final_2007_2015_removed_columns = df_final_2007_2015.drop(columns=['NotaProvaBrasil_MT_2005','NotaProvaBrasil_MT_2005','NotaProvaBrasil_LP_2005','NotaProvaBrasil_NotaMedia_2005','NotaProvaBrasil_MT_2007','NotaProvaBrasil_LP_2007','NotaProvaBrasil_NotaMedia_2007','NotaProvaBrasil_MT_2009','NotaProvaBrasil_LP_2009','NotaProvaBrasil_NotaMedia_2009','NotaProvaBrasil_MT_2011','NotaProvaBrasil_LP_2011','NotaProvaBrasil_NotaMedia_2011','NotaProvaBrasil_MT_2013','NotaProvaBrasil_LP_2013','NotaProvaBrasil_NotaMedia_2013','NotaProvaBrasil_MT_2015','NotaProvaBrasil_LP_2015','NotaProvaBrasil_NotaMedia_2015','NotaProvaBrasil_MT_2017','NotaProvaBrasil_LP_2017','NotaProvaBrasil_NotaMedia_2017','IndicadorRendimento_2005','IndicadorRendimento_2007','IndicadorRendimento_2009','IndicadorRendimento_2011','IndicadorRendimento_2013','IndicadorRendimento_2015','IndicadorRendimento_2017','Ideb2005','Ideb2007','Ideb2009','Ideb2011','Ideb2013','Ideb2015','Ideb2017'])
df_final_2007_2015_removed_columns

,ano_censo,cod_escola,cod_municipio,municipio,regiao,unidade_federativa,Rede,pib,pop,num_estudantes_ensino_fund_anos_finais,num_professores_em_regencia_fund_af,in_agua_filtrada,in_agua_inexistente,in_energia_rede_publica,in_energia_inexistente,in_esgoto_rede_publica,in_esgoto_inexistente,in_sala_professor,in_laboratorio_informatica,in_laboratorio_ciencias,in_sala_atendimento_especial,in_quadra_esportes,in_cozinha,in_biblioteca,in_sala_leitura,in_biblioteca_sala_leitura,in_parque_infantil,in_bercario,in_banheiro_fora_predio,in_banheiro_dentro_predio,in_secretaria,in_refeitorio,in_auditorio,in_patio_coberto,in_patio_descoberto,in_alojam_aluno,in_alojam_professor,in_area_verde,in_lavanderia,num_salas_utilizadas,num_equip_tv,num_equip_videocassete,num_equip_dvd,num_equip_parabolica,num_equip_copiadora,num_equip_retroprojetor,num_equip_impressora,num_equip_impressora_mult,num_equip_som,num_equip_multimidia,num_equip_fax,num_equip_foto,num_computador,num_comp_administrativo,num_comp_aluno,in_internet,num_funcionarios,aluno_prof_ratio,NotaProvaBrasil_MT_current_year,NotaProvaBrasil_LP_current_year,NotaProvaBrasil_NotaMedia_current_year,IndicadorRendimento__currrent_year,Ideb_current_year,Ideb_past_2_years,Ideb_diff,Ideb_next_2_years
0,2007,11000910,1100205,Porto Velho,Norte,Rondônia,Estadual,4474811.525,387181.0,538.0,21.0,1,0,1,0,0,0,1,1,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,51.0,25.619048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2
1,2007,12001139,1200203,Cruzeiro do Sul,Norte,Acre,Estadual,490308.260,76723.0,72.0,8.0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.0,9.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.7
2,2007,12020192,1200401,Rio Branco,Norte,Acre,Estadual,3406219.813,315529.0,65.0,7.0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31.0,9.285714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007,12009164,1200013,Acrelândia,Norte,Acre,Estadual,77839.731,11747.0,51.0,4.0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26.0,12.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007,12001015,1200203,Cruzeiro do Sul,Norte,Acre,Estadual,490308.260,76723.0,104.0,5.0,1,0,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,6.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27.0,20.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2007,12011460,1200401,Rio Branco,Norte,Acre,NaN,3406219.813,315529.0,114.0,5.0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25.0,22.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2007,13033611,1301902,Itacoatiara,Norte,Amazonas,Estadual,576408.798,81007.0,106.0,7.0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,16.0,15.142857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.4
7,2007,13024205,1302504,Manacapuru,Norte,Amazonas,Estadual,371972.016,80047.0,104.0,5.0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.0,20.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.7
8,2007,13012568,1300029,Alvarães,Norte,Amazonas,Estadual,45151.309,13268.0,443.0,14.0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34.0,31.642857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2007,13024213,1302504,Manacapuru,Norte,Amazonas,Estadual,371972.016,80047.0,172.0,10.0,1,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17.0,17.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.9


In [7]:
# leave only the completed rows (all columns with values)

df_final_2007_2015_removed_columns_without_null = df_final_2007_2015_removed_columns.dropna()
df_final_2007_2015_removed_columns_without_null

,ano_censo,cod_escola,cod_municipio,municipio,regiao,unidade_federativa,Rede,pib,pop,num_estudantes_ensino_fund_anos_finais,num_professores_em_regencia_fund_af,in_agua_filtrada,in_agua_inexistente,in_energia_rede_publica,in_energia_inexistente,in_esgoto_rede_publica,in_esgoto_inexistente,in_sala_professor,in_laboratorio_informatica,in_laboratorio_ciencias,in_sala_atendimento_especial,in_quadra_esportes,in_cozinha,in_biblioteca,in_sala_leitura,in_biblioteca_sala_leitura,in_parque_infantil,in_bercario,in_banheiro_fora_predio,in_banheiro_dentro_predio,in_secretaria,in_refeitorio,in_auditorio,in_patio_coberto,in_patio_descoberto,in_alojam_aluno,in_alojam_professor,in_area_verde,in_lavanderia,num_salas_utilizadas,num_equip_tv,num_equip_videocassete,num_equip_dvd,num_equip_parabolica,num_equip_copiadora,num_equip_retroprojetor,num_equip_impressora,num_equip_impressora_mult,num_equip_som,num_equip_multimidia,num_equip_fax,num_equip_foto,num_computador,num_comp_administrativo,num_comp_aluno,in_internet,num_funcionarios,aluno_prof_ratio,NotaProvaBrasil_MT_current_year,NotaProvaBrasil_LP_current_year,NotaProvaBrasil_NotaMedia_current_year,IndicadorRendimento__currrent_year,Ideb_current_year,Ideb_past_2_years,Ideb_diff,Ideb_next_2_years
80,2007,26129388,2613701,São Lourenço da Mata,Nordeste,Pernambuco,Estadual,303798.187,98744.0,436.0,25.0,1,0,1,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,20.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,67.0,17.440000,221.56,221.13,4.044833,0.738576,3.0,2.4,0.6,2.8
81,2007,26113929,2609600,Olinda,Nordeste,Pernambuco,Estadual,2211472.013,371596.0,1391.0,35.0,1,0,1,0,1,0,1,0,1,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,19.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,63.0,39.742857,202.5,195.38,3.298000,0.547975,1.8,1.5,0.3,3.0
204,2007,21007438,2107506,Paço do Lumiar,Nordeste,Maranhão,Municipal,253129.375,97724.0,336.0,16.0,1,0,1,0,1,0,1,1,1,0,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,53.0,21.000000,246.3,223.21,4.491833,0.810475,3.6,3.6,0.0,3.5
223,2007,26107546,2603454,Camaragibe,Nordeste,Pernambuco,Estadual,511051.527,139085.0,288.0,13.0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,37.0,22.153846,219.15,209.16,3.805167,0.563514,2.1,1.4,0.7,2.3
224,2007,26129132,2613701,São Lourenço da Mata,Nordeste,Pernambuco,Estadual,303798.187,98744.0,233.0,12.0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,36.0,19.416667,214.67,213.09,3.796000,0.862582,3.3,2.9,0.4,3.6
225,2007,26108801,2607901,Jaboatão dos Guararapes,Nordeste,Pernambuco,Estadual,5305071.070,621367.0,499.0,26.0,1,0,1,0,1,0,1,1,1,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,52.0,19.192308,229.15,223.88,4.217167,0.607656,2.6,2.1,0.5,2.8
1426,2007,26114178,2609600,Olinda,Nordeste,Pernambuco,Municipal,2211472.013,371596.0,255.0,7.0,1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,13.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,49.0,36.428571,198.1,193.35,3.190833,0.509825,1.6,2.3,-0.7,1.7
2825,2007,23144475,2309508,Orós,Nordeste,Ceará,Municipal,78394.887,21413.0,120.0,4.0,1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26.0,30.000000,199.06,189.43,3.141500,0.890593,2.8,3.0,-0.2,3.3
2840,2007,23144939,2311355,Quixelô,Nordeste,Ceará,Municipal,51825.162,15048.0,143.0,9.0,1,0,1,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,38.0,15.888889,241.88,230.41,4.538167,0.943999,4.3,3.8,0.5,3.9
2942,2007,23163666,2304202,Crato,Nordeste,Ceará,Municipal,606691.963,117311.0,566.0,23.0,1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,18.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,70.0,24.608696,212.94,202.87,3.596833,0.720525,2.6,2.8,-0.2,3.2


In [8]:
# saving dataset ready for training

df_final_2007_2015_removed_columns_without_null.to_csv('dataset_X_y.csv', index=False)